In [1]:
from ai_cdss.utils import MultiKeyDict
from rgs_interface.data.interface import fetch_patients_by_name, fetch_patients

In [35]:
mapping = {
    "VDH001": "juan",
    "VDH002": "ana",
    "VDH003": "jesus",
    "VDH004": "susana",
    "VDH005": "joan",
    "VDH006": "ismael",
    "VDH007": "anton",
    "VDH008": "gloria",
    "VDH009": "carles",
    "VDH010": "ricard",
    "VDH011": "rosacano",
    "VDH012": "rafael",
    "VDH013": "alfredo",
    "VDH014": "jose",
    "VDH015": "godo",
    "VDH016": "Hassan",
    "VDH017": "Carmen",
    # "VDH018": "juanfran",
    # "VDH019": "Marisa",
    # "VDH020": "mcarmen",
    "IT01" : "user_it01",
    "IT02" : "user_it02",
    "IT03" : "user_it03",
    "IT04" : "user_it04",
    "IT05" : "user_it05",
    "IT06" : "user_it06",
    "IT07" : "user_it07",
    "IT08" : "user_it08",
    "IT09" : "user_it09",
    "IT10" : "user_it10",
    "IT_02A" : "user_it02a",
    "IT_03A" : "user_it03a",
    "IT_04A" : "user_it04a",
    "IT_05A" : "user_it05a",
    "IT_06A" : "user_it06a",
    "IT_07A" : "user_it07a",
    "IT_08A" : "user_it08a",
    "IT_09A" : "user_it09a" ,
    "IT_10A" : "user_it10a" ,
}
map_id = {k:v for k,v in zip(mapping.values(), mapping.keys())}

In [36]:
patients = fetch_patients()

Database engine created successfully
Database engine closed


In [37]:
patients_nest = patients[patients["PATIENT_USER"].isin(mapping.values())][["PATIENT_ID", "PATIENT_USER"]]
patients_nest["NEST_ID"] = patients_nest["PATIENT_USER"].map(map_id)
map_nest_id = patients_nest[["PATIENT_ID", "NEST_ID"]].set_index("NEST_ID")
map_nest_id = map_nest_id.to_dict()["PATIENT_ID"]

In [43]:
map_nest_id.values()

dict_values([204, 775, 787, 788, 946, 947, 953, 955, 957, 1123, 1169, 1170, 1171, 1172, 1173, 1222, 1551, 1553, 1555, 1556, 2110, 2913, 2925, 2926, 2937, 2955, 2956, 2957, 2958, 2959, 2960, 2961, 2962, 2963, 3210, 3213])

### Read Clinical Scores

In [38]:
import pandas as pd

In [60]:
# Load from excel
column_names = ["BARTHEL", "ASH_PROXIMAL", "MA_DISTAL", "FATIGUE", "VAS", "FM_A", "FM_B", "FM_C", "FM_D", "FM_TOTAL", "ACT_AU", "ACT_QOM"]
scores = [pd.read_excel("data/NEST_clinical_score.xlsx", skiprows=1, sheet_name=time, index_col=0) for time in ["T0", "T1"]]
result = []
for i, score in enumerate(scores):
    score.columns = column_names
    score = score[score.index.isin(mapping.keys())]
    score.index = score.index.map(map_nest_id)
    score["FM_TOTAL"] = score[["FM_A", "FM_B", "FM_C", "FM_D"]].sum(axis=1)
    score["ACT_AU"].fillna(value=score["ACT_AU"].mean(), inplace=True)
    score["ACT_QOM"].fillna(value=score["ACT_QOM"].mean(), inplace=True)
    result.append(score)
    if i==0:
        score.to_csv(f"data/clinical_scores.csv", index_label="PATIENT_ID")

/tmp/ipykernel_3150512/2054862316.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  score["FM_TOTAL"] = score[["FM_A", "FM_B", "FM_C", "FM_D"]].sum(axis=1)
/tmp/ipykernel_3150512/2054862316.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  score["ACT_AU"].fillna(value=score["ACT_AU"

In [61]:
from ai_cdss.ppf import load_patient_subscales
load_patient_subscales("data/clinical_scores.csv")

File copied to default directory: /home/dav/.ai_cdss/data/clinical_scores.csv


,BARTHEL,ASH_PROXIMAL,MA_DISTAL,FATIGUE,VAS,FM_A,FM_B,FM_C,FM_D,FM_TOTAL,ACT_AU,ACT_QOM
PATIENT_ID,,,,,,,,,,,,
953,55,2,2,57,5.0,27,3,8,0,38,3.075123,3.107547
955,85,2,1,39,0.0,25,5,12,2,44,3.075123,3.107547
946,100,2,1,31,4.0,25,5,11,4,45,3.075123,3.107547
947,80,1,1,45,7.0,36,10,14,5,65,3.075123,3.107547
957,75,2,2,51,3.0,20,4,4,4,32,3.075123,3.107547
1555,100,1,1,44,3.0,35,6,14,4,59,3.075123,3.107547
1551,55,3,2,47,2.0,17,2,3,3,25,3.075123,3.107547
1553,100,1,2,52,8.0,19,5,8,2,34,3.075123,3.107547
1556,100,1,1,50,4.0,26,7,12,6,51,3.075123,3.107547


In [70]:
nest = pd.DataFrame({"DIFF": result[1]["FM_TOTAL"] - result[0]["FM_TOTAL"]})

In [71]:
nest["T0"] = result[0]["FM_TOTAL"]
nest["T1"] = result[1]["FM_TOTAL"]

In [72]:
nest.columns

Index(['DIFF', 'T0', 'T1'], dtype='object')

In [73]:
nest

,DIFF,T0,T1
953,0,38,38
955,0,44,44
946,1,45,46
947,0,65,65
957,0,32,32
1555,0,59,59
1551,0,25,25
1553,0,34,34
1556,0,51,51
1222,0,64,64


In [74]:
nest.to_csv("data/nest_clinical.csv", index_label="PATIENT_ID")

In [ ]:
map_nest_id

In [ ]:
import seaborn as sns
sns.scatterplot(nest, x="T0", y="T1")